In [1]:
import ibmJupyterNotebookStyles
ibmJupyterNotebookStyles.apply_ibm_styles()

# Quantum neural networks
## Quantum generative adversarial networks (qGAN)
This module explains the theory of qGANs.<sup>[1, 2]</sup> The next module illustrates their implementation.

### Introduction to GAN
Generative adversarial networks (GANs) are generative models that create new data instances that resemble your training data. One such example is the production of images that look like human faces, even though the faces do not belong to any real person. 

GANs achieve this by pairing a generator, which learns to create the target output, with a discriminator, which learns to distinguish true data from the output of the generator. The generator tries to fool the discriminator, while the discriminator tries to prevent being fooled.

### Generative models and discriminative models
A formal definition of generative models and discriminative models follows.

Given a set of data instances $X$ and a set of labels $Y$:
- Generative models capture the joint probability $p(X, Y)$, or just $p(X)$ if there are no labels.
- Discriminative models capture the conditional probability $p(Y | X)$.

More informally:
- Generative models can model a distribution of data in the space by creating new "fake" data instances that look like they are drawn from a real distribution.
- Discriminative models try to draw boundaries in the data space.

For example, the following diagram shows discriminative and generative models of handwritten digits:

![handwritten_digits](figs/generative_v_discriminative.png)

Figure 1: Discriminative and generative models of handwritten digits.

Here the discriminative model tries to tell the difference between handwritten 0's and 1's by drawing a line in the data space. If it gets the line right, it can distinguish 0's from 1's without ever having to model exactly where the instances are placed in the data space on either side of the line.

In contrast, the generative model tries to produce convincing 1's and 0's by generating digits that fall close to their real counterparts in the data space. It needs to capture correlations and model the complicated distribution throughout the data space.

### The basic structure of a GAN
A GAN consists of a generator and discriminator pair:
- The generator learns to generate convincing data. The generated instances become negative training examples for the discriminator.
- The discriminator learns to distinguish the generator's fake data from real data. The discriminator penalizes the generator for producing unconvincing results.

![gan_structure](figs/gan_diagram.svg)
Figure 2: The basic structure of a GAN.

At the beginning of training, the fake data produced by the generator is so obvious that the discriminator quickly identifies it as fake.
As training progresses, the generator begins to produce output that gets better at fooling the discriminator.
Finally, if generator training goes well, the discriminator gets worse at classifying real and fake data and its accuracy decreases.

The generator output is connected directly to the discriminator input. Through backpropagation, the discriminator's classification provides feedback that the generator uses to update its weights.

### Train the discriminator
The discriminator in a GAN is simply a classifier. It tries to distinguish real data from the fake data produced by the generator.

![discriminator_training](figs/gan_diagram_discriminator.svg)
Figure 3: Backpropagation in discriminator training. 

The discriminator's training data comes from two sources:
- Real data instances. The discriminator uses these instances as positive examples during training.
- Fake data instances created by the generator. The discriminator uses these instances as negative examples during training.

In Figure 3, the two "Sample" boxes represent these two data sources feeding into the discriminator. During discriminator training, the generator's weights remain constant while it produces fake data.

The discriminator connects to two loss functions: generator loss and discriminator loss. During training, the discriminator ignores the generator loss and uses only the discriminator loss.

In summary, during discriminator training:
1. The discriminator classifies both real data from input data and fake data from the generator.
2. The discriminator loss penalizes the discriminator for misclassifying a real instance as fake, or a fake instance as real.
3. The discriminator updates its weights through backpropagation from the discriminator loss through the discriminator network.

### Train the generator
The generator part of a GAN learns to create fake data by incorporating feedback from the discriminator and adjusting its weights. It learns to make the discriminator classify its output as real.

![generator_training](figs/gan_diagram_generator.svg)
Figure 4: Backpropagation in generator training.

Generator training requires tighter integration between the generator and the discriminator than discriminator training requires. The portion of the GAN that trains the generator includes:

- Random input, which is used so the GAN can produce a wide variety of data by sampling from different places in the target distribution.
- Generator network, which transforms the random input into a data instance.
- Discriminator network, which classifies the generated data.
- Discriminator output.
- Generator loss, which penalizes the generator for failing to fool the discriminator.

To train a generator, we adjust its weights to reduce the loss of its output. We observe that the generator is not directly connected to the loss, but it feeds into the discriminator net, and the discriminator produces the desired output. The generator loss penalizes the generator for producing a sample that the discriminator network classifies as fake. This whole network is included in backpropagation. Backpropagation adjusts each weight in the right direction by calculating the weight's impact on the output. The impact of a generator weight depends on the impact of the discriminator weights into which it feeds; therefore, backpropagation starts at the output and flows back through the discriminator into the generator.

In summary, we train the generator with the following procedure:
1. Sample random noise.
2. Produce generator output from sampled random noise.
3. Get discriminator "real" or "fake" classification for generator output.
4. Calculate loss from discriminator classification.
5. Backpropagate through both the discriminator and generator to obtain gradients.
6. Use gradients to change only the generator weights.

### Loss functions  
GANs mimic a probability distribution. They should therefore use loss functions that reflect the distance between the distribution of the data generated by the GAN and the distribution of the real data. 

#### Statistical measures
We first introduce two statistical measures that are used to identify the similarity between two probability distributions.

The Kolmogorov-Smirnov statistic is based on the (empirical) cumulative distribution functions 
$P (X \leq x)$ and $Q (X \leq x)$ and is given by:
$$ D_{KS} (P || Q) = \sup_{x \in X} |P(X \leq x) - Q(X \leq x)|. $$

Another measure is the Kullback-Leibler divergence (KL divergence)<sup>[3]</sup>. The KL divergence, also known as relative entropy, measures how one probability distribution $p$ diverges from a second expected probability distribution $q$. 

The KL divergence between two probability distributions $p(x)$ and $q(x)$ is given by:
$$ D_{KL}(p || q) = \sum_{x \in X} p(x) \log \frac{p(x)}{q(x)}. $$

The KL divergence is non-negative and will be zero, or minimum, when $p(x)$ is equal to $q(x)$ at every other point.

The KL divergence of two random distributions is an expected value, so it matters which distribution you use to take the expectation. Due to the asymmetric nature of the KL divergence, that is $D_{KL}(p || q) \neq D_{KL}(q || p)$; in general, it should not be used as a distance metric between two probability distributions.

#### GAN loss functions
Finding good GAN loss functions that capture the difference between two distributions is an area of active research, and many approaches have been proposed. 

Two common GAN loss functions are:
- (Modified) minimax loss
- Wasserstein loss

#### Minimax loss
"Minimax loss" is introduced in the original paper by Goodfellow et al.,<sup>[4]</sup> where the generator tries to minimize the following function while the discriminator tries to maximize it:
$$\mathbb{E}_x[\log(D(x))] + \mathbb{E}_z[\log(1 - D(G(z)))].$$
Here:
- $D(x)$ is the discriminator's estimate of the probability that real data instance $x$ is real.
- $\mathbb{E}_x$ is the expected value over all real data instances.
- $G(z)$ is the generator's output when given noise $z$.
- $D(G(z))$ is the discriminator's estimate of the probability that a fake instance is real.
- $\mathbb{E}_z$ is the expected value over all random inputs to the generator (in effect, the expected value over all generated fake instances $G(z)$).

The formula derives from the cross-entropy between the real and generated distributions.
The generator cannot directly affect the $\log(D(x))$ term in the function, so for the generator, minimizing the loss is equivalent to minimizing $\log(1 - D(G(z)))$.

The authors note that the above minimax loss function can cause the GAN to get stuck in the early stages of GAN training when the discriminator's job is very easy. They therefore suggest modifying the generator loss so that the generator tries to maximize $\log D(G(z))$, which is called "modified minimax loss".

#### Wasserstein loss
"Wasserstein loss" depends on a modification of the GAN scheme (called Wasserstein GAN or WGAN<sup>[5]</sup>) in which the discriminator does not actually classify instances. For each instance it outputs a number. This number does not have to be less than one or greater than zero, so we can't use 0.5 as a threshold to decide whether an instance is real or fake. Discriminator training can only try to make the output bigger for real instances than for fake instances.

Because it cannot really discriminate between real and fake, the WGAN discriminator is actually called a "critic" instead of a "discriminator". This distinction has theoretical importance, but for practical purposes we can treat it as an acknowledgment that inputs to the loss functions do not need to be probabilities.

The loss functions have simple forms:

Critic Loss: $D(x) - D(G(z))$

The discriminator tries to maximize this function. In other words, it tries to maximize the difference between its output on real instances and its output on fake instances.

Generator Loss: $D(G(z))$

The generator tries to maximize this function. In other words, it tries to maximize the discriminator's output for its fake instances.

In these functions:
- $D(x)$ is the critic's output for a real instance.
- $G(z)$ is the generator's output when given noise $z$.
- $D(G(z))$ is the critic's output for a fake instance.
- The output of critic $D$ does not have to be between 1 and 0.
- The formulas derive from the earth mover's distance<sup>[6]</sup> between the real and generated distributions.

The theoretical justification for the Wasserstein GAN (or WGAN) requires that the weights throughout the GAN be clipped so that they remain within a constrained range.

Wasserstein GANs are less vulnerable to getting stuck than minimax-based GANs, and avoid problems with vanishing gradients. The earth mover's distance also has the advantage of being a true metric: a measure of distance in a space of probability distributions. Cross-entropy is not a metric in this sense.

### Training of the GAN as a whole 
Because a GAN contains two separately trained networks, its training algorithm must address two complications:
- GANs must juggle two different kinds of training (generator and discriminator).
- GAN convergence is hard to identify.

GAN training proceeds in alternating periods:
1. The discriminator trains for one or more epochs.
2. The generator trains for one or more epochs.
3. Repeat steps 1 and 2 to continue to train the generator and discriminator networks.

Note that the discriminator's training must be started with a much simpler classification problem. If you cannot train a classifier to tell the difference between real and fake data, even for the initial random generator output, you cannot start the GAN training.

Also note that as the generator improves with training, the discriminator performance gets worse because the discriminator cannot easily tell the difference between real and fake. If the generator succeeds perfectly, then the discriminator has a 50% accuracy, which is a totally random prediction.

This progression poses a problem for convergence of the GAN as a whole: the discriminator feedback gets less meaningful over time. If the GAN continues training past the point where the discriminator is giving completely random feedback, then the generator starts to train on junk feedback, and its own quality may collapse.

For a GAN, convergence is often a fleeting state, rather than stable one.

### Introduction to qGAN
Quantum GAN (qGAN) is a system where either the generator, the discriminator, or both are replaced with quantum systems<sup>[7]</sup>.
The first theoretical discussion of quantum GANs was followed by demonstrations of qGAN implementations. Some focused on quantum state estimation, i.e., finding a quantum channel whose output is an estimate to a given quantum state.
Others exploited qGANs to generate classical data samples in accordance with the training data’s underlying distribution.

Later in this section we discuss qGAN implementation that learns and loads probability distributions into quantum states. More specifically, the aim of the qGAN is not to produce classic samples in accordance with given classical training data, but to train the quantum generator to create a quantum state that represents the data’s underlying probability distribution. The resulting quantum channel, given by the quantum generator, enables efficient loading of an approximated probability distribution into a quantum state. It can be easily prepared and reused as often as needed. 

This notebook explained the theory of qGANs. Their implementation is described in the next notebook.

### References:
<a id='Zoufal2019'></a>1. Zoufal, Lucchi & Woerner, [Quantum Generative Adversarial Networks for learning and loading random distributions.](https://www.nature.com/articles/s41534-019-0223-2) *npj Quantum Information*, 2019. <br>
<a id='TensorFlow'></a>2. [Generative adversarial networks, Machine Learning Courses](https://developers.google.com/machine-learning/gan).  <br>
<a id='Ahirwar2019'></a>3. Ahirwar, *Generative Adversarial Networks Projects.* Packt Publishing, 2019.<br>
<a id='Goodfellow2014'></a>4. Ian J. Goodfellow, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville, Yoshua Bengio, [Generative Adversarial Networks](https://arxiv.org/abs/1406.2661).  <br>
<a id='Arjovsky2017'></a>5. Martin Arjovsky, Soumith Chintala, Léon Bottou, [Wasserstein GAN.](https://arxiv.org/abs/1701.07875) <br>
<a id='Levina2001'></a>6. Elizaveta Levina, Peter Bickel, "The Earth Mover's distance is the Mallows distance: some insights from statistics."  *Proceedings, Eighth IEEE International Conference on Computer Vision.* ICCV 2001, pp. 251-256 vol.2, doi: 10.1109/ICCV.2001.937632.<br>
<a id='Lloyd2018'></a>7. S. Lloyd, C. Weedbrook, [Quantum generative adversarial learning.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.121.040502) *Physical Review Letters,* 2018.  <br>

<span style="font-size:10pt; font-weight:bold;"> &copy; Copyright IBM Corp. 2023 <br/>This content is categorized as an “existing” IBM asset as part of the IBM Quantum Accelerator contract. </span>
